# <div align="left"> 🧬 Capstone: b-factor prediction for alphaFold structures for epitope prediction </div>

<img src="https://userguide.mdanalysis.org/stable/_images/rmsf-view.gif" height="256" align="right" style="height:256px">

#### Our goal is to leverage AlphaFold2 to improve epitope predictions beyond sequence-based methods through considering structural constraints on antigen processing, similar to the referenced paper.


<div align="left">
  <h3> 📁 Google Drive </h3>
</div>

Upon running this notebook, a new folder gets created in your Drive. You define the name of this folder and it will store all the data generated via this notebook.

<div align="left">
  <h3> 📖 Reference Materials </h3>
</div>

This notebook takes inspiration from these resources:

- [CD4+ T-cell Epitope Prediction Using Antigen Processing Constraints](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5321161/#SD1)
- [Highly accurate protein structure prediction with AlphaFold](https://doi.org/10.1038/s41586-021-03819-2)

<div align="left">
  <h3> 🌐 Legal & Data Formats </h3>
</div>

The provided code operates under the [Apache 2.0 license](https://www.apache.org/licenses/LICENSE-2.0). The license of the [structural prediction model parameters](https://github.com/deepmind/alphafold/#model-parameters-license) is Creative Commons Attribution 4.0 International ([CC BY 4.0](https://creativecommons.org/licenses/by/4.0/legalcode)).
For details regarding the PAE file format, consult the [AFDB FAQ](https://alphafold.ebi.ac.uk/faq/#faq-7).

# Step 1: Upload your gene list and generate structures

In [ ]:
#@title Set Up Google Drive


%pip install biopython --q
import os, requests
import pandas as pd
from Bio.PDB import PDBParser, PPBuilder
import warnings
from Bio import BiopythonWarning

class DataSheet:
    def __init__(self):
        self.dframe = pd.read_csv(os.path.join(project_dir, 'files', 'DataSheet.csv'), index_col='ID').drop("Unnamed: 0", axis=1)[['Antigen/Gene', 'PDB ID']]
        self.ids = [pdb_id for pdb_id in self.dframe['PDB ID'] if len(pdb_id) == 4]
        self.dframe = self.dframe[self.dframe['PDB ID'].isin(self.ids)]
        self.dframe['pdb path'] = None  # Add new column 'path' with default None
        self.load()

    def load(self):
        parser = PDBParser()
        ppb = PPBuilder()
        for pdb_id in self.ids:
            pdb_path = os.path.join("/content/drive/MyDrive/EpiFold2/", "files", "pdb files", f"{pdb_id}.pdb")
            if not os.path.exists(pdb_path):
                response = requests.get(f"https://files.rcsb.org/download/{pdb_id}.pdb")
                if response.status_code == 200:
                    pdb_path = os.path.join("/content/drive/MyDrive/EpiFold2/", "files", "pdb files", f"{pdb_id}.pdb")
                    if not os.path.exists(os.path.join("/content/drive/MyDrive/EpiFold2/", "files", "pdb files")):
                      os.mkdir(os.path.join("/content/drive/MyDrive/EpiFold2/", "files", "pdb files"))
                    with open(pdb_path, 'wb') as f:
                        f.write(response.content)
                else:
                    self.dframe.drop(self.dframe['PDB ID'] == pdb_id, axis=0)
            self.dframe.loc[self.dframe['PDB ID'] == pdb_id, 'pdb path'] = pdb_path

            fasta_sequence = self.extract_sequence(parser, ppb, pdb_path)
            self.dframe.loc[self.dframe['PDB ID'] == pdb_id, 'FastA'] = str(fasta_sequence)
        self.dframe = self.dframe.reset_index(drop=True)

    def extract_sequence(self, parser, ppb, pdb_file_path):
        structure = parser.get_structure('pdb', pdb_file_path)
        for pp in ppb.build_peptides(structure):
            return pp.get_sequence()

def custom_warning(message, category, filename, lineno, file=None, line=None):
    if "Chain" in str(message) and "is discontinuous" in str(message):
        return
    else:
        warnings.showwarning(message, category, filename, lineno, file, line)
warnings.showwarning = custom_warning

project_name = 'EpiFold2'
project_dir = f'/content/drive/MyDrive/{project_name}'

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

if not os.path.exists(project_dir):
    os.mkdir(project_dir)
    os.mkdir(os.path.join(project_dir, 'files'))
print("\nPlease save you datasheet as DataSheet.csv")
from google.colab import files
files.upload()
import shutil
current_colab_dir = f'/content/drive/MyDrive/{project_name}.ipynb'
source_dir = '/content/DatasetSheet.csv'
target_dir = f'/content/drive/MyDrive/{project_name}/files/DataSheet.csv'
shutil.move(source_dir, target_dir)
datasheet = DataSheet()


#-------------------------------------------------------------------------------------------------------------

df = datasheet.dframe
df[['PDB ID', 'FastA']]


def write_fasta(filename, sequence):
    with open(filename, 'w') as f:
        f.write(">protein\n")
        f.write(sequence)

fasta_files_dir = "/content/drive/MyDrive/EpiFold2/files/fasta files"
os.makedirs(fasta_files_dir, exist_ok=True)  # Creates the directory if it doesn't already exist

for pdb_id, seq in zip(df['PDB ID'], df['FastA']):
    write_fasta(os.path.join(fasta_files_dir, f"{pdb_id}.fasta"), seq)


input_dir = fasta_files_dir
result_dir = "/content/drive/MyDrive/EpiFold2/files/results"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.2 MB/s eta 0:00:00
Mounted at /content/drive

Please save you datasheet as DataSheet.csv


Saving DatasetSheet.csv to DatasetSheet.csv


In [ ]:
#@title Install AlphaFold2
%%bash -s $use_amber $use_templates $python_version

set -e

USE_AMBER=$1
USE_TEMPLATES=$2
PYTHON_VERSION=$3

if [ ! -f COLABFOLD_READY ]; then
  # install dependencies
  # We have to use "--no-warn-conflicts" because colab already has a lot preinstalled with requirements different to ours
  pip install -q --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold" "tensorflow-cpu==2.11.0"
  pip uninstall -yq jax jaxlib
  pip install -q "jax[cuda]==0.3.25" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
  touch COLABFOLD_READY
fi

# Download params (~1min)
python -m colabfold.download

# setup conda
if [ ${USE_AMBER} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f CONDA_READY ]; then
    wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Miniconda3-latest-Linux-x86_64.sh
    conda config --set auto_update_conda false
    touch CONDA_READY
  fi
fi
# setup template search
if [ ${USE_TEMPLATES} == "True" ] && [ ! -f HH_READY ]; then
  conda install -y -q -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 python="${PYTHON_VERSION}" 2>&1 1>/dev/null
  touch HH_READY
fi
# setup openmm for amber refinement
if [ ${USE_AMBER} == "True" ] && [ ! -f AMBER_READY ]; then
  conda install -y -q -c conda-forge openmm=7.7.0 python="${PYTHON_VERSION}" pdbfixer 2>&1 1>/dev/null
  touch AMBER_READY
fi



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.0/242.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.7 requires jax>=0.4.6, but you have jax 0.3.25 which is incompatible.
flax 0.7.4 requires jax>=0.4.2, but you have jax 0.3.25 which is incompatible.
orbax-checkpoint 0.4.1 requires jax>=0.4.9, but you have jax 0.3.25 which is incompatible.


In [ ]:
import sys
from colabfold.batch import get_queries, run
from colabfold.download import default_data_dir
from colabfold.utils import setup_logging
from pathlib import Path
import warnings
import logging

warnings.simplefilter(action='ignore', category=FutureWarning)
setup_logging(Path(result_dir).joinpath("log.txt"))
queries, is_complex = get_queries(input_dir)

run(
    queries=queries,
    result_dir=result_dir,
    use_templates=False,
    use_amber=False,
    msa_mode="MMseqs2 (UniRef+Environmental)",
    model_type="auto",
    num_models=1,
    num_recycles=3,
    model_order=[1],
    is_complex=is_complex,
    data_dir=default_data_dir,
    keep_existing_results=True,
    rank_by="auto",
    pair_mode="unpaired+paired",
    stop_at_score=98,
    zip_results=True,
    user_agent="colabfold/google-colab-batch"
)


--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/colabfold/batch.py", line 1297, in run
    jax.tools.colab_tpu.setup_tpu()
  File "/usr/local/lib/python3.10/dist-packages/jax/tools/colab_tpu.py", line 38, in setup_tpu
    colab_tpu_addr = os.environ['COLAB_TPU_ADDR'].split(':')[0]
  File "/usr/lib/python3.10/os.py", line 680, in __getitem__
    raise KeyError(key) from None
KeyError: 'COLAB_TPU_ADDR'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1104, in emit
    self.flush()
  File "/usr/lib/python3.10/logging/__init__.py", line 1084, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    e

KeyboardInterrupt: ignored

# Step 2: Generate Stability Profiles

## Setup (just run this all and move on)

In [ ]:
# this cell moves and renames the best structure predictions to a new folder in drive
import os
import zipfile
import shutil

def extract_best_pdb(zip_path, target_dir):
    pdb_id = os.path.basename(zip_path).split(".")[0]
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Extract the specific rank_001.pdb file
        for file in zip_ref.namelist():
            if "rank_001" in file and file.endswith(".pdb"):
                zip_ref.extract(file)
                # Copy the file to the target directory
                shutil.copy(file, os.path.join(target_dir, file))

def rename_files_in_folder(folder_path, prefix="AF-"):
    files = os.listdir(folder_path)
    for file_name in files:
        if file_name.endswith(".pdb"):
            old_file_path = os.path.join(folder_path, file_name)
            new_file_name = f"{prefix}{file_name.split('_')[0]}.pdb"
            new_file_path = os.path.join(folder_path, new_file_name)
            os.rename(old_file_path, new_file_path)


results_dir = '/content/drive/MyDrive/EpiFold2/files/results/'
target_dir = '/content/drive/MyDrive/EpiFold2/files/AlphaFoldStructures/'

os.makedirs(target_dir, exist_ok=True)

zip_paths = [os.path.join(results_dir, i) for i in os.listdir(results_dir) if '.zip' in i]
for zip_path in zip_paths:
    extract_best_pdb(zip_path, target_dir)

rename_files_in_folder(target_dir)

In [ ]:
# Here and below is the analyze_results.ipynb file I shared

from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.MMCIFParser import MMCIFParser
from collections import OrderedDict
import requests, os
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from Bio import BiopythonWarning
import warnings
from scipy.stats import zscore
import pandas as pd
import numpy as np

def get_structural_data(protein_file):
    if os.path.splitext(protein_file)[1] == '.pdb':
        parser = PDBParser(PERMISSIVE=1)
    else:
        raise ValueError(f"Unknown file format: {protein_file}")

    structure = parser.get_structure("test", protein_file)
    model = structure[0]
    try:
        chain = model["A"]
    except KeyError:
        for c in model:
            print(f"Weird chain id: {c.id}")
        chain = model[c.id]
    bfactors = OrderedDict()
    residues = OrderedDict()
    for residue in chain:
        if residue.has_id("N"):
            bfactors[residue.get_id()[1]] = residue["N"].get_bfactor()
            residues[residue.get_id()[1]] = residue.get_resname()
    return bfactors, residues

def calculate_zscores1(values):
    z_scores = zscore(values)
    # Scale values to fall within the range -1, 1
    scaler = MinMaxScaler(feature_range=(-1, 1))
    return scaler.fit_transform(z_scores.reshape(-1, 1)).ravel()

def calculate_zscores2(values):
    z_scores = zscore(values)
    mean_zscore = z_scores.mean()
    modified_z_scores = [mean_zscore + abs(zs - mean_zscore) * (1 if zs > mean_zscore else (-1))
                         for zs in z_scores]
    # Convert NaN values to numeric and replace with 0
    #modified_z_scores = np.nan_to_num(modified_z_scores)
    # Scale values to fall within the range -1, 1
    #scaler = MinMaxScaler(feature_range=(-1, 1))
    #return scaler.fit_transform(np.array(modified_z_scores).reshape(-1, 1)).ravel()
    return modified_z_scores

from sklearn.preprocessing import MinMaxScaler

def smooth_data(data, window_size):
    if window_size < 3 or window_size % 2 == 0:
        raise ValueError("'window_size' must be 1) an odd integer; and 2) greater than 1.")
    if not isinstance(data, pd.Series):
        data = pd.Series(data)
    smoothed_data = data.rolling(window_size, center=True).mean() # Moving window average
    for i in range(window_size//2): # take the average of available data (handles data at either end when complete window is not available).
        smoothed_data.iloc[i] = data.iloc[:i+window_size//2+1].mean()
        smoothed_data.iloc[-i-1] = data.iloc[-i-window_size//2-1:].mean()
    return smoothed_data.tolist()

def handle_warning(message, category, filename, lineno, file=None, line=None):
    # counts 'chain is discontinuous' warnings for each PDB ID.
    if issubclass(category, BiopythonWarning) and "Chain" in str(message) and "is discontinuous" in str(message):
        handle_warning.discontinuous_chain_warnings += 1
    else:
        # For all other warnings, use default behavior
        warnings.showwarning(message, category, filename, lineno, file, line)
warnings.showwarning = handle_warning
handle_warning.discontinuous_chain_warnings = 0

def process_upid(pdb_id):
    handle_warning.discontinuous_chain_warnings = 0
    af_file_path = f'/content/drive/MyDrive/EpiFold2/files/AlphaFoldStructures/AF-{pdb_id}.pdb'
    pdb_file_path = f'/content/drive/MyDrive/EpiFold2/files/pdb files/{pdb_id}.pdb'
    files = pdb_file_path, af_file_path
    #files = [os.path.expanduser(f'~/Desktop/EpiFold/files/pdb files/{i}{pdb_id}.{"pdb"}') for i in ['','AF-']]
    data_and_residues = [get_structural_data(file) for file in files]  # Use the new function here
    common_residues = sorted(set(data_and_residues[0][0]).intersection(set(data_and_residues[1][0])))
    num_mismatch = sum(data_and_residues[0][1][k] != data_and_residues[1][1][k]
                       for k in common_residues)

    print(f'PDB ID: {pdb_id}')
    print(f'\tsequence discontinuities: {handle_warning.discontinuous_chain_warnings}')
    print(f'\t{num_mismatch}/{len(common_residues)} residues do not match up')
    print(f'\tsequence length (X-ray): {len(data_and_residues[0][0])}\n\tsequence length (AlphaFoldDB): {len(data_and_residues[1][0])}\n\tcommon residues by index: {len(common_residues)}')
    # Z-score calculation for b-factors and plddt
    bfactors_smoothed_normalized = [
        (calculate_zscores2 if i == 0 else calculate_zscores1)(smooth_data([data[k] for k in common_residues], window_size=window_size)) * (-1 if i==1 else 1)
        for i, (data, window_size) in enumerate(zip([data_and_residues[0][0], data_and_residues[1][0]], [3, 3]))
    ]
    same_sign_pct = sum((b1>=0)==(b2>=0) for b1, b2 in zip(*bfactors_smoothed_normalized)) / len(bfactors_smoothed_normalized[0]) * 100
    print(f'\tPercentage of residues with same sign: {same_sign_pct:.2f}%\n')

    correlation, p_value = pearsonr(*bfactors_smoothed_normalized)
    avg_plddt = np.mean([data_and_residues[1][0][k] for k in common_residues])
    txt = "Correlation coefficient: {:.2f}\n p-value: {:.3e}".format(correlation, p_value)

    plt.figure(figsize=(12, 4))
    # Compute and plot medians
    medians = []
    for i, color in enumerate(['g', 'b']):
        median = np.median(bfactors_smoothed_normalized[i])
        #plt.axhline(y=median, color=color, linestyle='--')
        plt.plot(bfactors_smoothed_normalized[i], color=color)
        medians.append(median)

    # Plot the differences between the medians
    #plt.axhline(y=0, color='b', linestyle='--')
    plt.title(f"{pdb_id} b-factors v.s. p-LDDT")
    plt.xlabel('Residue index')
    plt.ylabel('Normalized value')
    plt.show()



    plt.scatter(*bfactors_smoothed_normalized)
    plt.title(pdb_id)
    plt.xlabel('b-factor')
    plt.ylabel('p-LDDT')



    plt.gca().text(1.05, 0.95, txt, transform=plt.gca().transAxes,
                   fontsize=8, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    plt.subplots_adjust(right=0.7)
    plt.show()
    return p_value, correlation, avg_plddt

from Bio.Align import PairwiseAligner
def aligned(pdb_id):
    af_file_path = f'/content/drive/MyDrive/EpiFold2/files/AlphaFoldStructures/AF-{pdb_id}.pdb'
    pdb_file_path = f'/content/drive/MyDrive/EpiFold2/files/pdb files/{pdb_id}.pdb'
    af_bfactors, af_residues = get_structural_data(af_file_path)
    pdb_bfactors, pdb_residues = get_structural_data(pdb_file_path)
    af_sequence = ''.join(af_residues.values())
    pdb_sequence = ''.join(pdb_residues.values())
    aligner = PairwiseAligner()
    alignments = aligner.align(af_sequence, pdb_sequence)
    best_alignment = alignments[0]
    aligned_af_sequence = str(best_alignment[0])
    aligned_pdb_sequence = str(best_alignment[1])
    #print(len(af_sequence) - len(aligned_af_sequence), len(pdb_sequence) - len(aligned_pdb_sequence))
    return [af_sequence, aligned_af_sequence], [pdb_sequence, aligned_pdb_sequence], [list(af_bfactors.values()), list(pdb_bfactors.values())]

In [ ]:
def smooth_graph(graph, n):
    smoothed_graph = []
    for i in range(len(graph)):
        start_index = max(0, i - (n // 2))
        end_index = min(len(graph), i + (n // 2) + 1)
        window = graph[start_index:end_index]
        smoothed_value = np.mean(window)
        smoothed_graph.append(smoothed_value)
    return smoothed_graph

In [ ]:
from numpy.core.overrides import array_function_dispatch
%pip install freesasa --q
fs = [i.split('.')[0] for i in os.listdir('/content/drive/MyDrive/EpiFold2/files/pdb files/') if not 'AF' in i and '.pdb' in i]
results_list = []
for protein_name in fs:
    try:
        res = aligned(protein_name)

        af, bf = res[2][0], res[2][1]
        diffsaf =  af
        diffspdb = bf
        window = 25
        saf, sbf = smooth_graph(af, window), smooth_graph(bf, window)

        dbf, daf = array_function_dispatch, bf

        from scipy.stats import linregress
        slope, intercept, r_value, p_value, std_err = linregress(dbf, daf)
        results_list.append([protein_name, r_value, p_value])
    except:
        results_list.append([protein_name, np.NAN, np.NAN])



from Bio.PDB.PDBParser import PDBParser
import freesasa
import matplotlib.pyplot as plt

def get_asa(pdb_file):
    return [x.relativeTotal for x in freesasa.calc(freesasa.Structure(pdb_file)).residueAreas()['A'].values()]

## Do analysis

In [ ]:
#@title Aggregating Inputs: Experimental v.s. AlphaFold2

results_lst = [i for i in results_list if str(i[2]) != 'nan']
results_lst_sort = sorted(results_lst, key=lambda x: x[1])
good_ids = [i[0] for i in results_lst_sort]

accumulated = []

for good_id in good_ids:
    res = aligned(good_id)
    af, bf = res[2][0], res[2][1]

    window = 15
    asa = get_asa(f'/content/drive/MyDrive/EpiFold2/files/pdb files/{good_id}.pdb')
    asa2 = get_asa(f'/content/drive/MyDrive/EpiFold2/files/AlphaFoldStructures/AF-{good_id}.pdb')
    asa = [-1*i for i in asa]
    asa2 = [-1*i for i in asa2]
    ssasa, ssasa2, saf, sbf = smooth_graph(asa, window), smooth_graph(asa2, window), smooth_graph(af, window), smooth_graph(bf, window)
    sbf = [-1*i for i in sbf]

    ssasa, ssasa2, saf, sbf = calculate_zscores2(ssasa), calculate_zscores2(ssasa2), calculate_zscores2(saf), calculate_zscores2(sbf)
    plt.figure(figsize=(12, 8))
    plt.plot(saf, label='AlphaFold2 p-LDDT')
    plt.plot(ssasa2,label='AlphaFold2 Available Surface Area')
    afaverage = [(i + j)/2 for i, j in zip(saf, ssasa2)]
    afaverage = calculate_zscores2(afaverage)


    expaverage = [(i + j)/2 for i, j in zip(sbf, ssasa)]
    expaverage = calculate_zscores2(expaverage)


    average = [(i + j + k)/3 for i, j ,k in zip(saf, sbf, ssasa)]

    from scipy.stats import linregress
    slope, intercept, r_value, p_value, std_err = linregress(expaverage, afaverage)


    print(good_id)
    print("Slope:", slope)
    print("Intercept:", intercept)
    print("R-squared:", r_value**2)
    print("p-value:", p_value)
    print('')


    plt.plot(afaverage, label='Average')
    plt.legend(loc='best')
    plt.xlabel('Residue Index')
    plt.ylabel('z-score')
    plt.axhline(y=0, c='b')
    plt.show()

    plt.figure(figsize=(12, 8))
    plt.plot(sbf, label='Experimental b-factors')
    plt.plot(ssasa,label='Experimental Available Surface Area')


    plt.plot(expaverage, label='Average')
    plt.legend(loc='best')
    plt.xlabel('Residue Index')
    plt.ylabel('z-score')
    plt.axhline(y=0, c='b')
    plt.show()



    plt.figure(figsize=(12, 8))
    plt.scatter(afaverage, expaverage)
    plt.xlabel('Crystal Stability Profile')
    plt.ylabel('AlphaFold2 Stability Profile')
    plt.show()


#RMSF
The root-mean-square fluctuation (RMSF) of a structure is the time average of the RMSD. It is calculated according to the below equation, where $x_i$ represents the coordinates of particle $i$, and $\langle x_i \rangle$ is the ensemble average position of $i$.

$p_{i}^{\text{RMSF}} = \sqrt{\langle (x_i - \langle x_i \rangle)^2 \rangle}$


#b-factors
The B-factor, or Debye-Waller factor, is derived from the root-mean-square fluctuation (RMSF) and provides information about the mobility or flexibility of atoms in a protein structure. It is calculated using the square of the RMSF values and can be used to visualize the regions of the protein that are more rigid or flexible.

$B_i = \frac{8\pi^2}{3}\cdot(p_{i}^{\text{RMSF}})^2$


#RMSF to b-factor
Question: Can we predict/approximate b-factors using MD simlations on AF structions? (See [Becksteinlab github](https://github.com/Becksteinlab/AdKGromacsTutorial/blob/master/docs/analysis/rmsf.rst))

$B_i = \frac{8\pi^2\cdot\langle (x_i - \langle x_i \rangle)^2 \rangle}{3}$




In [ ]:
!pip install mdanalysis --q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 5.5 MB/s eta 0:00:00


In [ ]:
average.results['positions']

array([[-14.99999905,  -6.20300007,  -7.98399973],
       [-11.79699993,  -3.91199994,  -7.67199993],
       [-11.63300037,  -1.495     , -10.57800007],
       ...,
       [ 22.15600014,  -3.01600003, -17.34399986],
       [ 22.42200089,   0.396     , -15.75      ],
       [ 25.98399925,   1.85899985, -16.53100014]])

# Overview


## RMSD and RMSF

The root-mean-square-fluctuation (RMSF) of an amino acid on a protein uses the below equation, where $x_{i}$ is the coordinates of particle $i$, and $⟨x_{i}⟩$ is the average position of $i$ during a molecular dynamics simulation.
$p_{i} = \sqrt{\langle (x_i - \langle x_i \rangle)^2 \rangle}$



In [ ]:
from MDAnalysis.analysis import align, rms
import MDAnalysis as mda

# Load the Universe from a PDB file
u = mda.Universe("/content/AF-1Z7H.pdb")

# Create average structure
average = align.AverageStructure(u, u, select='protein and name CA', ref_frame=0).run()
ref = average.results.universe

# Align trajectory to the average structure
aligner = align.AlignTraj(u, ref, select='protein and name CA', in_memory=True).run()

# Select Calpha atoms
c_alphas = u.select_atoms('protein and name CA')

# Calculate RMSF
rmsf_analysis = rms.RMSF(c_alphas).run()

# Print RMSF per residue
for residue, rmsf_value in zip(c_alphas.residues, rmsf_analysis.rmsf):
    print(f"Residue: {residue.resid}, RMSF: {rmsf_value}")

Residue: 1, RMSF: 0.0
Residue: 2, RMSF: 0.0
Residue: 3, RMSF: 0.0
Residue: 4, RMSF: 0.0
Residue: 5, RMSF: 0.0
Residue: 6, RMSF: 0.0
Residue: 7, RMSF: 0.0
Residue: 8, RMSF: 0.0
Residue: 9, RMSF: 0.0
Residue: 10, RMSF: 0.0
Residue: 11, RMSF: 0.0
Residue: 12, RMSF: 0.0
Residue: 13, RMSF: 0.0
Residue: 14, RMSF: 0.0
Residue: 15, RMSF: 0.0
Residue: 16, RMSF: 0.0
Residue: 17, RMSF: 0.0
Residue: 18, RMSF: 0.0
Residue: 19, RMSF: 0.0
Residue: 20, RMSF: 0.0
Residue: 21, RMSF: 0.0
Residue: 22, RMSF: 0.0
Residue: 23, RMSF: 0.0
Residue: 24, RMSF: 0.0
Residue: 25, RMSF: 0.0
Residue: 26, RMSF: 0.0
Residue: 27, RMSF: 0.0
Residue: 28, RMSF: 0.0
Residue: 29, RMSF: 0.0
Residue: 30, RMSF: 0.0
Residue: 31, RMSF: 0.0
Residue: 32, RMSF: 0.0
Residue: 33, RMSF: 0.0
Residue: 34, RMSF: 0.0
Residue: 35, RMSF: 0.0
Residue: 36, RMSF: 0.0
Residue: 37, RMSF: 0.0
Residue: 38, RMSF: 0.0
Residue: 39, RMSF: 0.0
Residue: 40, RMSF: 0.0
Residue: 41, RMSF: 0.0
Residue: 42, RMSF: 0.0
Residue: 43, RMSF: 0.0
Residue: 44, RMSF: 0

# Overview


## RMSD and RMSF

The root-mean-square-fluctuation (RMSF) of a structure is the time average of the RMSD.

$B_i = \frac{8\pi^2}{3} \cdot (p_{i}^{\text{RMSF}})^2$

RMSF
It is calculated according to the below equation, where xi is the coordinates of particle i, and ⟨xi⟩ is the ensemble average position of i.
$p_{i} = \sqrt{\langle (x_i - \langle x_i \rangle)^2 \rangle}$






    - Root Mean Square Deviation measures how structures and parts of structures change between frames.
    - RSMD is typically plotted vs time.
    - A leveling off or flattening of this curve can indicate that the protein has equilibrated.
    - RMSF is computed as the square root of the variance of the fluctuation around the average position.
RMSF is computed over a series of time steps of a molecular dynamics simulation.
    

## RMSF


    - Root Mean Square Fluctuation is a calculation of how much a particular residue moves (fluctuates) during a simulation.
    - RMSF is typically plotted vs. residue number
    - RSMF can indicate structurally which amino acids in a protein contribute the most to a molecular motion.